In [1]:
using FFTW, Random, Test
using MacroTools: prettify

In [2]:
Random.seed!(1234)
sample_vector_input = rand(32);
sample_tuple_input = Tuple(sample_vector_input);

In [3]:
function test_fft(fft)
    @test FFTW.fft(sample_vector_input) ≈ fft(sample_tuple_input)
end

test_fft (generic function with 1 method)

## Работа с комплексными числами

In [4]:
function fft_rec_c(a)
    n = length(a)
    @assert n > 0 && ispow2(n)

    b = Vector{Complex{Float64}}(undef, n)

    if isone(n)
        b[1] = a[1]
        return b
    end

    n2 = n ÷ 2
    w = exp(-2 * pi * im / n)

    a1 = [a[k] for k in 1:2:n]
    a2 = [a[k] for k in 2:2:n]
    y1 = fft_rec_c(a1)
    y2 = fft_rec_c(a2)
    for k in 1:n2
        t = w^(k - 1) * y2[k]
        b[k] = y1[k] + t
        b[k+n2] = y1[k] - t
    end
    return b
end

fft_rec_c (generic function with 1 method)

In [5]:
fft_rec_c((10.0, 20.0, 30.0, 40.0))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [6]:
test_fft(fft_rec_c)

Test Passed

## Переход к действительным числам

In [7]:
function fft_rec_(a)
    n = length(a)
    b_r = Vector{Float64}(undef, n)
    b_i = Vector{Float64}(undef, n)

    if isone(n)
        b_r[1] = a[1]
        b_i[1] = 0.0
        return b_r, b_i
    end

    n2 = n ÷ 2

    a1 = [a[k] for k in 1:2:n]
    a2 = [a[k] for k in 2:2:n]
    y1_r, y1_i = fft_rec_(a1)
    y2_r, y2_i = fft_rec_(a2)

    for k in 1:n2
        theta = -2 * (k - 1) * pi / n
        wk_r, wk_i = cos(theta), sin(theta)

        t_r = wk_r * y2_r[k] - wk_i * y2_i[k]
        t_i = wk_r * y2_i[k] + wk_i * y2_r[k]
        b_r[k] = y1_r[k] + t_r
        b_i[k] = y1_i[k] + t_i
        b_r[k+n2] = y1_r[k] - t_r
        b_i[k+n2] = y1_i[k] - t_i
    end
    return b_r, b_i
end

fft_rec_ (generic function with 1 method)

In [8]:
function fft_rec(a)
    n = length(a)
    @assert n > 0 && ispow2(n)
    b_r, b_i = fft_rec_(a)
    return [complex(b_r[k], b_i[k]) for k in 1:n]
end

fft_rec (generic function with 1 method)

In [9]:
fft_rec((10.0, 20.0, 30.0, 40.0))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [10]:
test_fft(fft_rec)

Test Passed

## Переделываем вычислитель в генератор

In [11]:
function fft_gen_impl1!(a, es, m)
    n = length(a)

    b_r = [Symbol("b_r_", m[], "_", k) for k in 1:n]
    b_i = [Symbol("b_i_", m[], "_", k) for k in 1:n]
    m[] += 1

    if isone(n)
        append!(es, (
            :($(b_r[1]) = $(a[1])),
            :($(b_i[1]) = 0.0)
        ))
        return b_r, b_i
    end

    n2 = n ÷ 2
    a1 = [a[k] for k in 1:2:n]
    a2 = [a[k] for k in 2:2:n]
    y1_r, y1_i = fft_gen_impl1!(a1, es, m)
    y2_r, y2_i = fft_gen_impl1!(a2, es, m)

    w = exp(-2 * pi * im / n)

    for k in 1:n2
        t_r = Symbol("t_r_", m[])
        t_i = Symbol("t_i_", m[])
        m[] += 1

        theta = -2 * (k - 1) * pi / n
        wk_r, wk_i = cos(theta), sin(theta)

        append!(es, (
            :($t_r = $wk_r * $(y2_r[k]) - $wk_i * $(y2_i[k])),
            :($t_i = $wk_r * $(y2_i[k]) + $wk_i * $(y2_r[k])),
            :($(b_r[k]) = $(y1_r[k]) + $t_r),
            :($(b_i[k]) = $(y1_i[k]) + $t_i),
            :($(b_r[k+n2]) = $(y1_r[k]) - $t_r),
            :($(b_i[k+n2]) = $(y1_i[k]) - $(t_i))
        ))
    end


    return b_r, b_i
end

fft_gen_impl1! (generic function with 1 method)

In [12]:
function fft_gen_impl1(n)
    @assert n > 0 && ispow2(n)

    a = [:(a[$k]) for k in 1:n]
    es = Expr[]
    m = Ref(1)
    b_r, b_i = fft_gen_impl1!(a, es, m)

    b = [:(complex($(b_r[k]), $(b_i[k]))) for k in 1:n]
    quote
        $(es...)
        return $(Expr(:vect, b...))
    end

end

fft_gen_impl1 (generic function with 1 method)

In [13]:
fft_gen_impl1(4) |> prettify

quote
    b_r_3_1 = a[1]
    b_i_3_1 = 0.0
    b_r_4_1 = a[3]
    b_i_4_1 = 0.0
    t_r_5 = 1.0b_r_4_1 - 0.0b_i_4_1
    t_i_5 = 1.0b_i_4_1 + 0.0b_r_4_1
    b_r_2_1 = b_r_3_1 + t_r_5
    b_i_2_1 = b_i_3_1 + t_i_5
    b_r_2_2 = b_r_3_1 - t_r_5
    b_i_2_2 = b_i_3_1 - t_i_5
    b_r_7_1 = a[2]
    b_i_7_1 = 0.0
    b_r_8_1 = a[4]
    b_i_8_1 = 0.0
    t_r_9 = 1.0b_r_8_1 - 0.0b_i_8_1
    t_i_9 = 1.0b_i_8_1 + 0.0b_r_8_1
    b_r_6_1 = b_r_7_1 + t_r_9
    b_i_6_1 = b_i_7_1 + t_i_9
    b_r_6_2 = b_r_7_1 - t_r_9
    b_i_6_2 = b_i_7_1 - t_i_9
    t_r_10 = 1.0b_r_6_1 - 0.0b_i_6_1
    t_i_10 = 1.0b_i_6_1 + 0.0b_r_6_1
    b_r_1_1 = b_r_2_1 + t_r_10
    b_i_1_1 = b_i_2_1 + t_i_10
    b_r_1_3 = b_r_2_1 - t_r_10
    b_i_1_3 = b_i_2_1 - t_i_10
    t_r_11 = 6.123233995736766e-17b_r_6_2 - -1.0b_i_6_2
    t_i_11 = 6.123233995736766e-17b_i_6_2 + -1.0b_r_6_2
    b_r_1_2 = b_r_2_2 + t_r_11
    b_i_1_2 = b_i_2_2 + t_i_11
    b_r_1_4 = b_r_2_2 - t_r_11
    b_i_1_4 = b_i_2_2 - t_i_11
    return [complex(b_r_1_1,

In [14]:
@generated function fft_gen1(a::NTuple{N,Float64}) where {N}
    fft_gen_impl1(N)
end

fft_gen1 (generic function with 1 method)

In [15]:
fft_gen1((10.0, 20.0, 30.0, 40.0))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [16]:
test_fft(fft_gen1)

Test Passed

## Замена приближенных числе на точные.</br>Обеспечение того, чтобы $sin (π / 4) = cos (π / 4)$.

In [17]:
function wp(n, j)
    if j == 0
        (1.0, 0.0)
    elseif 2 * j == n
        (-1.0, 0.0)
    elseif 4 * j == n
        (0.0, -1.0)
    elseif 4 * j == 3 * n
        (0.0, 1.0)
    elseif mod(8 * j, n) == 0
        @assert isodd((8 * j) ÷ n)
        quadrant = 4 - ((((8 * j) ÷ n) - 1) ÷ 2)
        cos_signs = 1.0, -1.0, -1.0, 1.0
        sin_signs = 1.0, 1.0, -1.0, -1.0
        csh = cos(pi / 4.0)
        csh * cos_signs[quadrant], csh * sin_signs[quadrant]
    else
        theta = (-2 * j) * pi / n
        cos(theta), sin(theta)
    end
end

wp (generic function with 1 method)

In [18]:
[wp(16, j) for j in 0:15]

16-element Vector{Tuple{Float64, Float64}}:
 (1.0, 0.0)
 (0.9238795325112867, -0.3826834323650898)
 (0.7071067811865476, -0.7071067811865476)
 (0.38268343236508984, -0.9238795325112867)
 (0.0, -1.0)
 (-0.3826834323650897, -0.9238795325112867)
 (-0.7071067811865476, -0.7071067811865476)
 (-0.9238795325112867, -0.3826834323650899)
 (-1.0, 0.0)
 (-0.9238795325112868, 0.38268343236508967)
 (-0.7071067811865476, 0.7071067811865476)
 (-0.38268343236509034, 0.9238795325112865)
 (0.0, 1.0)
 (0.38268343236509, 0.9238795325112866)
 (0.7071067811865476, 0.7071067811865476)
 (0.9238795325112865, 0.3826834323650904)

In [19]:
function fft_gen_impl2!(a, es, m)
    n = length(a)

    b_r = [Symbol("b_r_", m[], "_", k) for k in 1:n]
    b_i = [Symbol("b_i_", m[], "_", k) for k in 1:n]
    m[] += 1

    if isone(n)
        append!(es, (
            :($(b_r[1]) = $(a[1])),
            :($(b_i[1]) = 0.0)
        ))
        return b_r, b_i
    end

    n2 = n ÷ 2
    a1 = [a[k] for k in 1:2:n]
    a2 = [a[k] for k in 2:2:n]
    y1_r, y1_i = fft_gen_impl2!(a1, es, m)
    y2_r, y2_i = fft_gen_impl2!(a2, es, m)

    for k in 1:n2
        t_r = Symbol("t_r_", m[])
        t_i = Symbol("t_i_", m[])
        m[] += 1

        wk_r, wk_i = wp(n, k - 1)
        append!(es, (
            :($t_r = $wk_r * $(y2_r[k]) - $wk_i * $(y2_i[k])),
            :($t_i = $wk_r * $(y2_i[k]) + $wk_i * $(y2_r[k])),
            :($(b_r[k]) = $(y1_r[k]) + $t_r),
            :($(b_i[k]) = $(y1_i[k]) + $t_i),
            :($(b_r[k+n2]) = $(y1_r[k]) - $t_r),
            :($(b_i[k+n2]) = $(y1_i[k]) - $(t_i))
        ))
    end

    return b_r, b_i
end

fft_gen_impl2! (generic function with 1 method)

In [20]:
function fft_gen_impl2(n)
    @assert n > 0 && ispow2(n)

    es = Expr[]
    m = Ref(1)
    a = [:(a[$k]) for k in 1:n]
    b_r, b_i = fft_gen_impl2!(a, es, m)

    b = [:(complex($(b_r[k]), $(b_i[k]))) for k in 1:n]
    quote
        $(es...)
        return $(Expr(:vect, b...))
    end

end

fft_gen_impl2 (generic function with 1 method)

In [21]:
fft_gen_impl2(4) |> prettify

quote
    b_r_3_1 = a[1]
    b_i_3_1 = 0.0
    b_r_4_1 = a[3]
    b_i_4_1 = 0.0
    t_r_5 = 1.0b_r_4_1 - 0.0b_i_4_1
    t_i_5 = 1.0b_i_4_1 + 0.0b_r_4_1
    b_r_2_1 = b_r_3_1 + t_r_5
    b_i_2_1 = b_i_3_1 + t_i_5
    b_r_2_2 = b_r_3_1 - t_r_5
    b_i_2_2 = b_i_3_1 - t_i_5
    b_r_7_1 = a[2]
    b_i_7_1 = 0.0
    b_r_8_1 = a[4]
    b_i_8_1 = 0.0
    t_r_9 = 1.0b_r_8_1 - 0.0b_i_8_1
    t_i_9 = 1.0b_i_8_1 + 0.0b_r_8_1
    b_r_6_1 = b_r_7_1 + t_r_9
    b_i_6_1 = b_i_7_1 + t_i_9
    b_r_6_2 = b_r_7_1 - t_r_9
    b_i_6_2 = b_i_7_1 - t_i_9
    t_r_10 = 1.0b_r_6_1 - 0.0b_i_6_1
    t_i_10 = 1.0b_i_6_1 + 0.0b_r_6_1
    b_r_1_1 = b_r_2_1 + t_r_10
    b_i_1_1 = b_i_2_1 + t_i_10
    b_r_1_3 = b_r_2_1 - t_r_10
    b_i_1_3 = b_i_2_1 - t_i_10
    t_r_11 = 0.0b_r_6_2 - -1.0b_i_6_2
    t_i_11 = 0.0b_i_6_2 + -1.0b_r_6_2
    b_r_1_2 = b_r_2_2 + t_r_11
    b_i_1_2 = b_i_2_2 + t_i_11
    b_r_1_4 = b_r_2_2 - t_r_11
    b_i_1_4 = b_i_2_2 - t_i_11
    return [complex(b_r_1_1, b_i_1_1), complex(b_r_1_2, b_i_1_2)

In [22]:
@generated function fft_gen2(a::NTuple{N,Float64}) where {N}
    fft_gen_impl2(N)
end

fft_gen2 (generic function with 1 method)

In [23]:
fft_gen2((10.0, 20.0, 30.0, 40.0))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [24]:
test_fft(fft_gen2)

Test Passed

## Упрощение выражений через переписывание (Metatheory.jl)

In [25]:
using Metatheory, Metatheory.Rewriters

In [26]:
opt_rules = @theory c c1 c2 x y begin

    x::Float64 + y::Float64 => x + y
    x + 0.0 --> x
    0.0 + y --> y

    x::Float64 - y::Float64 => x - y
    x - 0.0 --> x
    0.0 - y --> -y

    x::Float64 * y::Float64 => x * y
    x * 0.0 --> 0.0
    0.0 * y --> 0.0

    x * 1.0 --> x
    1.0 * y --> y
    x * -1.0 --> -x
    -1.0 * y --> -y

    x + (-y) --> x - y
    x - (-y) --> x + y

    c::Float64 * x + c::Float64 * y --> c * (x + y)
    c::Float64 * x - c::Float64 * y --> c * (x - y)

    c1::Float64 * x + c2::Float64 * y => :($c1 * ($x - $y)) where {c1==-c2}
    c1::Float64 * x - c2::Float64 * y => :($c1 * ($x + $y)) where {c1==-c2}

    c1::Float64 * x + c2::Float64 * y => :($c1 * ($x + $(c2 / c1) * $y))
    c1::Float64 * x - c2::Float64 * y => :($c1 * ($x - $(c2 / c1) * $y))

end;

In [27]:
strategy = (#= Fixpoint ∘ =# Postwalk ∘ Chain)
opt_expr(e) = strategy(opt_rules)(e)

opt_expr (generic function with 1 method)

In [28]:
@test opt_expr(:(2.0 + 3.0)) == 5.0
@test opt_expr(:(u + 0.0)) == :u
@test opt_expr(:(0.0 + v)) == :v
@test opt_expr(:(2.0 - 3.0)) == -1.0
@test opt_expr(:(u - 0.0)) == :u
@test opt_expr(:(0.0 - v)) == :(-v)
@test opt_expr(:(2.0 * 3.0)) == 6.0
@test opt_expr(:(u * 0.0)) == 0.0
@test opt_expr(:(0.0 * v)) == 0.0
@test opt_expr(:(u * 1.0)) == :u
@test opt_expr(:(1.0 * v)) == :v
@test opt_expr(:(u * -1.0)) == :(-u)
@test opt_expr(:(-1.0 * v)) == :(-v)
@test opt_expr(:(u + (-v))) == :(u - v)
@test opt_expr(:(u - (-v))) == :(u + v)
@test opt_expr(:(10.0 * u + 10.0 * v)) == :(10.0 * (u + v))
@test opt_expr(:(10.0 * u - 10.0 * v)) == :(10.0 * (u - v))
@test opt_expr(:(10.0 * u + -10.0 * v)) == :(10.0 * (u - v))
@test opt_expr(:(10.0 * u - -10.0 * v)) == :(10.0 * (u + v))
@test opt_expr(:(10.0 * u + 20.0 * v)) == :(10.0 * (u + 2.0 * v))
@test opt_expr(:(10.0 * u - 20.0 * v)) == :(10.0 * (u - 2.0 * v))

Test Passed

In [29]:
function do_ass_o!(es, d, s::Symbol, c::Float64)
    d[s] = c
end

function do_ass_o!(es, d, s::Symbol, u::Symbol)
    d[s] = u
end

function do_ass_o!(es, d, s::Symbol, e)
    d[s] = s
    push!(es, :($s = $e))
end

do_ass_o! (generic function with 3 methods)

In [30]:
function ass_o!(es, d, s::Symbol, e)
    do_ass_o!(es, d, s, opt_expr(e))
end

ass_o! (generic function with 1 method)

In [31]:
function fft_gen_impl_o!(a, d, es, m)
    n = length(a)

    b_r = [Symbol("b_r_", m[], "_", k) for k in 1:n]
    b_i = [Symbol("b_i_", m[], "_", k) for k in 1:n]
    m[] += 1

    if isone(n)
        ass_o!(es, d, b_r[1], a[1])
        ass_o!(es, d, b_i[1], 0.0)
        return [d[b_r[1]]], [d[b_i[1]]]
    end

    n2 = n ÷ 2
    a1 = [a[k] for k in 1:2:n]
    a2 = [a[k] for k in 2:2:n]

    y1_r, y1_i = fft_gen_impl_o!(a1, d, es, m)
    y2_r, y2_i = fft_gen_impl_o!(a2, d, es, m)

    for k in 1:n2
        t_r = Symbol("t_r_", m[])
        t_i = Symbol("t_i_", m[])
        m[] += 1

        (wk_r, wk_i) = wp(n, k - 1)
        ass_o!(es, d, t_r,
            :($wk_r * $(y2_r[k]) - $wk_i * $(y2_i[k])))
        ass_o!(es, d, t_i,
            :($wk_r * $(y2_i[k]) + $wk_i * $(y2_r[k])))
        ass_o!(es, d, b_r[k],
            :($(y1_r[k]) + $(d[t_r])))
        ass_o!(es, d, b_i[k],
            :($(y1_i[k]) + $(d[t_i])))
        ass_o!(es, d, b_r[k+n2],
            :($(y1_r[k]) - $(d[t_r])))
        ass_o!(es, d, b_i[k+n2],
            :($(y1_i[k]) - $(d[t_i])))
    end

    d_b_r = [d[b_r[k]] for k in 1:n]
    d_b_i = [d[b_i[k]] for k in 1:n]
    return d_b_r, d_b_i
end

fft_gen_impl_o! (generic function with 1 method)

In [32]:
function fft_gen_impl_o(n)
    @assert n > 0 && ispow2(n)

    a = [:(a[$k]) for k in 1:n]
    d = Dict{Symbol,Any}()
    es = Expr[]
    m = Ref(1)

    b_r, b_i = fft_gen_impl_o!(a, d, es, m)

    b = [:(complex($(b_r[k]), $(b_i[k]))) for k in 1:n]

    quote
        $(es...)
        return $(Expr(:vect, b...))
    end
end

fft_gen_impl_o (generic function with 1 method)

In [33]:
fft_gen_impl_o(16) |> prettify

quote
    b_r_5_1 = a[1]
    b_r_6_1 = a[9]
    b_r_4_1 = b_r_5_1 + b_r_6_1
    b_r_4_2 = b_r_5_1 - b_r_6_1
    b_r_9_1 = a[5]
    b_r_10_1 = a[13]
    b_r_8_1 = b_r_9_1 + b_r_10_1
    b_r_8_2 = b_r_9_1 - b_r_10_1
    b_r_3_1 = b_r_4_1 + b_r_8_1
    b_r_3_3 = b_r_4_1 - b_r_8_1
    t_i_13 = -b_r_8_2
    b_i_3_4 = -t_i_13
    b_r_16_1 = a[3]
    b_r_17_1 = a[11]
    b_r_15_1 = b_r_16_1 + b_r_17_1
    b_r_15_2 = b_r_16_1 - b_r_17_1
    b_r_20_1 = a[7]
    b_r_21_1 = a[15]
    b_r_19_1 = b_r_20_1 + b_r_21_1
    b_r_19_2 = b_r_20_1 - b_r_21_1
    b_r_14_1 = b_r_15_1 + b_r_19_1
    b_r_14_3 = b_r_15_1 - b_r_19_1
    t_i_24 = -b_r_19_2
    b_i_14_4 = -t_i_24
    b_r_2_1 = b_r_3_1 + b_r_14_1
    b_r_2_5 = b_r_3_1 - b_r_14_1
    t_r_26 = 0.7071067811865476 * (b_r_15_2 + t_i_24)
    t_i_26 = 0.7071067811865476 * (t_i_24 - b_r_15_2)
    b_r_2_2 = b_r_4_2 + t_r_26
    b_i_2_2 = t_i_13 + t_i_26
    b_r_2_6 = b_r_4_2 - t_r_26
    b_i_2_6 = t_i_13 - t_i_26
    t_i_27 = -b_r_14_3
    b_i_2_7 = -t_i_27

In [34]:
@generated function fft_gen_o(a::NTuple{N,Float64}) where {N}
    fft_gen_impl_o(N)
end

fft_gen_o (generic function with 1 method)

In [35]:
fft_gen_o((10.0, 20.0, 30.0, 40.0))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [36]:
test_fft(fft_gen_o)

Test Passed